In [17]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import ModelCheckpoint
import os

folder_path = './'
# Utwórz pustą ramkę danych
merged_df = pd.DataFrame()

# Pętla po plikach w katalogu
for filename in os.listdir(folder_path):
    if filename.endswith("_merged.csv"):
        file_path = os.path.join(folder_path, filename)
        current_df = pd.read_csv(file_path)
        
        # Połącz z wcześniej wczytanymi danymi
        merged_df = pd.concat([merged_df, current_df], ignore_index=True)

data_csv = merged_df[['Date', 'Close', 'compound', 'likes', 'retweets', 'risk_of_PAD']]



# Przygotowanie danych
data_csv = data_csv.drop('Date', axis=1)
data_csv = data_csv.reset_index(drop=True)
T = data_csv.values
T = T.astype('float32')

# Okno czasowe
window_size = 20

# Tworzenie funkcji do tworzenia cech
def create_features(data, window_size):
    X, Y = [], []
    for i in range(len(data) - window_size):
        window = data[i:(i + window_size), :4]
        target = data[i + window_size, -1]  # tylko kolumna 'risk_of_pad'
        X.append(window)
        Y.append(target)
    return np.array(X), np.array(Y)

# Tworzenie cech dla wszystkich dostępnych danych
X, Y = create_features(T, window_size)

print("Shape of X before reshape:", X.shape)
# Reshape do formatu [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
print("Shape of X after reshape:", X.shape)



# Model
# model = Sequential()
# model.add(LSTM(units=50, activation='sigmoid', input_shape=(X.shape[1], X.shape[2])))
# model.add(Dropout(0.2))
# model.add(Dense(1))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model = Sequential()
model.add(LSTM(units=100, activation='relu', input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=100, activation='relu'))  # Nowa warstwa LSTM model.add(LSTM(units=100, activation='relu', input_shape=(X.shape[1], X.shape[2]))) 
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Zapis modelu
filepath = 'saved_models_final2/model_epoch_{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
history = model.fit(X, Y, epochs=400, batch_size=20, callbacks=[checkpoint], verbose=1, shuffle=False)


Shape of X before reshape: (2081, 20, 4)
Shape of X after reshape: (2081, 20, 4)
Epoch 1/400
102/105 [============================>.] - ETA: 0s - loss: 359.3538 - accuracy: 0.7779
Epoch 1: loss improved from inf to 354.44748, saving model to saved_models_final2/model_epoch_01.hdf5
105/105 [==============================] - 4s 15ms/step - loss: 354.4475 - accuracy: 0.7794
Epoch 2/400
  7/105 [=>............................] - ETA: 1s - loss: 0.0630 - accuracy: 0.9857

/home/rudospiew/.local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


103/105 [============================>.] - ETA: 0s - loss: 147.7821 - accuracy: 0.7039
Epoch 2: loss improved from 354.44748 to 146.89900, saving model to saved_models_final2/model_epoch_02.hdf5
105/105 [==============================] - 2s 15ms/step - loss: 146.8990 - accuracy: 0.7059
Epoch 3/400
104/105 [============================>.] - ETA: 0s - loss: 111.2115 - accuracy: 0.7312
Epoch 3: loss improved from 146.89900 to 111.22974, saving model to saved_models_final2/model_epoch_03.hdf5
105/105 [==============================] - 2s 14ms/step - loss: 111.2297 - accuracy: 0.7309
Epoch 4/400
102/105 [============================>.] - ETA: 0s - loss: 103.6639 - accuracy: 0.6446
Epoch 4: loss improved from 111.22974 to 102.40932, saving model to saved_models_final2/model_epoch_04.hdf5
105/105 [==============================] - 1s 14ms/step - loss: 102.4093 - accuracy: 0.6454
Epoch 5/400
103/105 [============================>.] - ETA: 0s - loss: 112.5835 - accuracy: 0.7447
Epoch 5: loss di